In [44]:
import io # For reading files
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import os

In [45]:
path = 'emails/' 

In [46]:
dataFiles = []
for root, folder, files in os.walk(path):
    for file in files:
        dataFiles.append(root+'/'+file)

In [47]:
file = open(dataFiles[2])
data = file.read()
print(data)
file.close()

From timc@2ubh.com  Thu Aug 22 13:52:59 2002
Return-Path: <timc@2ubh.com>
Delivered-To: zzzz@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id 0314547C66
	for <zzzz@localhost>; Thu, 22 Aug 2002 08:52:58 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:52:59 +0100 (IST)
Received: from n16.grp.scd.yahoo.com (n16.grp.scd.yahoo.com
    [66.218.66.71]) by dogma.slashnull.org (8.11.6/8.11.6) with SMTP id
    g7MCrdZ07070 for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:53:39 +0100
X-Egroups-Return: sentto-2242572-52733-1030020820-zzzz=spamassassin.taint.org@returns.groups.yahoo.com
Received: from [66.218.67.198] by n16.grp.scd.yahoo.com with NNFMP;
    22 Aug 2002 12:53:40 -0000
X-Sender: timc@2ubh.com
X-Apparently-To: zzzzteana@yahoogroups.com
Received: (EGP: mail-8_1_0_1); 22 Aug 2002 12:53:39 -0000
Received: (qma

In [48]:
# Yield does not end the function rather allows to make it iterable whereas return ends the flow of function
def read_file(path):
    for root, folder, files in os.walk(path):
        for file in files:
            filename = root+"/"+file
            file = io.open(filename,encoding="latin1")
            body =False
            data = []
            lines = file.readlines()
            for line in lines:
                if line == "\n":
                    body = True
                if body:
                    data.append(line)
            file.close()
            msg = "\n".join(data)
            yield msg, file
                

In [49]:
def df(path, classification):
    rows =[]
    index= []
    for message ,filename in read_file(path):
        rows.append({"message":message,"class":classification})
        index.append(filename)
    return pd.DataFrame(data = rows , index = index)
dataset = pd.DataFrame({"message": [], "class":[]})
dataset = dataset.append(df("emails/ham","ham"))
dataset = dataset.append(df("emails/spam","spam"))


In [50]:
dataset.head()

,class,message
<_io.TextIOWrapper name='emails/ham/00001.7c53336b37003a9286aba55d2945844c' mode='r' encoding='latin1'>,ham,"\n\n Date: Wed, 21 Aug 2002 10:54:46..."
<_io.TextIOWrapper name='emails/ham/00002.9c4069e25e1ef370c078db7ee85ff9ac' mode='r' encoding='latin1'>,ham,"\n\nMartin A posted:\n\nTassos Papadopoulos, t..."
<_io.TextIOWrapper name='emails/ham/00003.860e3c3cee1b42ead714c5c874fe25f7' mode='r' encoding='latin1'>,ham,\n\nMan Threatens Explosion In Moscow \n\n\n\n...
<_io.TextIOWrapper name='emails/ham/00004.864220c5b6930b209cc287c361c99af1' mode='r' encoding='latin1'>,ham,\n\nKlez: The Virus That Won't Die\n\n \n\nAlr...
<_io.TextIOWrapper name='emails/ham/00005.bf27cdeaf0b8c4647ecd61b1d09da613' mode='r' encoding='latin1'>,ham,"\n\n> in adding cream to spaghetti carbonara,..."


In [90]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [91]:
msg=dataset['message'][0]

In [92]:
import re

In [93]:
tokens = []
for i in range(len(dataset)):
    msg = re.findall('[\w]+',dataset['message'][i])
    tokens.append(msg)

In [94]:
def textProcessing(df):
    tokens=[]
    for i in range(len(df)):
        msg = re.findall('[\w]+',df['message'][i])
        tokens.append(msg)
    stopwordsList = stopwords.words("english")
    
    wordsList = []
    for tokenList in tokens:
        words = []
        for word in tokenList:
            if word.lower() not in stopwordsList:
                words.append(word.lower())
        wordsList.append(words)
                
    wnet = WordNetLemmatizer()
    for i in range(len(wordsList)):
        for j in range(len(wordsList[i])):
            wordsList[i][j] = wnet.lemmatize(wordsList[i][j], pos ="v")
            
    for i in range(len(wordsList)):
        wordsList[i] = " ".join(wordsList[i])
        
    return wordsList
    
          

In [95]:
wordsList = textProcessing(dataset)

In [96]:
wordsList[1]

'martin post tassos papadopoulos greek sculptor behind plan judge limestone mount kerdylio 70 miles east salonika far mount athos monastic community ideal patriotic sculpture well alexander granite feature 240 ft high 170 ft wide museum restore amphitheatre car park admire crowd plan mountain limestone granite limestone weather pretty fast yahoo group sponsor 4 dvds free p join http us click yahoo com pt6ybb nxieaa mg3haa 7gsolb tm unsubscribe group send email forteana unsubscribe egroups com use yahoo group subject http docs yahoo com info term'

In [97]:
cv = CountVectorizer()

In [98]:
vect = cv.fit_transform(wordsList)

In [99]:
vect =vect.toarray()

In [100]:
y = dataset["class"]

In [101]:
encoder = LabelEncoder()

In [102]:
y = encoder.fit_transform(y)

In [103]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [104]:
x_train, x_test, y_train, y_test = train_test_split(vect, y, test_size=0.25)

In [105]:
x_train.shape

(2250, 58388)

In [106]:
x_test.shape

(750, 58388)

In [107]:
y_train.shape

(2250,)

In [108]:
y_test.shape

(750,)

In [109]:
reg = LogisticRegression()
reg.fit(x_train, y_train)

C:\Users\ASHISH RANA\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [110]:
y_pred = reg.predict(x_test)

In [111]:
accuracy_score(y_test, y_pred)

0.9826666666666667

In [112]:
nb = GaussianNB()

In [113]:
nb.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [114]:
y_pred_2 = nb.predict(x_test)

In [115]:
accuracy_score(y_test, y_pred_2)

0.9733333333333334